In [1]:
import numpy as np
import math
from keras import optimizers
from utils import *
from model import *
import matlab.engine

Using TensorFlow backend.


# Initialize the model

In [2]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"

# Initialize the model on X,Y,Z on UMassd-Kinect Dataset

In [3]:
def one_obj2(frame_l=16, joint_n=25, joint_d=3):

    input_joints = Input(name='joints', shape=(frame_l, joint_n, joint_d))
    input_joints_diff = Input(name='joints_diff', shape=(frame_l, joint_n, joint_d))
    
    ##########branch 1##############
    x = Conv2D(filters = 32, kernel_size=(1,1),padding='same')(input_joints)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    
    x = Conv2D(filters = 16, kernel_size=(3,1),padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)

    x = Permute((1,3,2))(x)
    
    x = Conv2D(filters = 16, kernel_size=(3,3),padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)   
    ##########branch 1##############
    
    ##########branch 2##############Temporal difference
    x_d = Conv2D(filters = 32, kernel_size=(1,1),padding='same')(input_joints_diff)
    x_d = BatchNormalization()(x_d)
    x_d = LeakyReLU()(x_d)
    
    x_d = Conv2D(filters = 16, kernel_size=(3,1),padding='same')(x_d)
    x_d = BatchNormalization()(x_d)
    x_d = LeakyReLU()(x_d)

    x_d = Permute((1,3,2))(x_d)
    
    x_d = Conv2D(filters = 16, kernel_size=(3,3),padding='same')(x_d)
    x_d = BatchNormalization()(x_d)
    x_d = LeakyReLU()(x_d)
    ##########branch 2##############
    
    x = concatenate([x,x_d],axis=-1)
    
    x = Conv2D(filters = 32, kernel_size=(1,1),padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = MaxPool2D(pool_size=(2, 2))(x) 
    x = Dropout(0.1)(x)
       
    x = Conv2D(filters = 64, kernel_size=(1,1),padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = MaxPool2D(pool_size=(2, 2))(x) 
    x = Dropout(0.1)(x)
      
    model = Model([input_joints,input_joints_diff],x)

    return model


def multi_obj2(frame_l=16, joint_n=25, joint_d=3):
    inp_j_0 = Input(name='inp_j_0', shape=(frame_l, joint_n, joint_d))
    
    inp_j_diff_0 = Input(name='inp_j_diff_0', shape=(frame_l, joint_n, joint_d))
    
    
    inp_j_1 = Input(name='inp_j_1', shape=(frame_l, joint_n, joint_d))
    inp_j_diff_1 = Input(name='inp_j_diff_1', shape=(frame_l, joint_n, joint_d))
    
    single = one_obj2()
    x_0 = single([inp_j_0,inp_j_diff_0])
    x_1 = single([inp_j_1,inp_j_diff_1])
      
    x = Maximum()([x_0,x_1])
    
    x = Flatten()(x)
    x = Dropout(0.1)(x)
     
    x = Dense(256)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU()(x)
    x = Dropout(0.1)(x)
    
    x = Dense(6, activation='sigmoid')(x)
      
    model = Model([inp_j_0,inp_j_diff_0,inp_j_1,inp_j_diff_1],x)
    
    return model


lr=0.001
adam = optimizers.Adam(lr)
model2 = multi_obj2()
model2.compile(adam, loss='mean_squared_error')
model2.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inp_j_0 (InputLayer)            (None, 16, 25, 3)    0                                            
__________________________________________________________________________________________________
inp_j_diff_0 (InputLayer)       (None, 16, 25, 3)    0                                            
__________________________________________________________________________________________________
inp_j_1 (InputLayer)            (None, 16, 25, 3)    0                                            
__________________________________________________________________________________________________
inp_j_diff_1 (InputLayer)       (None, 16, 25, 3)    0                                            
__________________________________________________________________________________________________
model_1 (M

# Train the model on X,Y,Z on UMassd-Kinect Dataset

In [4]:
def zoom1(p):
    l = p.shape[0]
    p_new = np.empty([16,25,3]) 
    for m in range(25):
        for n in range(3):
            p_new[:,m,n] = inter.zoom(p[:,m,n],16/l)[:16]
    return p_new


epochs = 400
for e in range(epochs):
    X_0 = []
    X_1 = []
    X_2 = []
    X_3 = []
    Y = []
    i = -1
    for root, dirs, files in os.walk("data/train"):
        if(i < 6):
            i = i+1
        for file in files:
            if file.endswith(".txt"):
                skeleton_data = pd.read_csv(os.path.join(root, file), sep=" ", header=None)
                p_0 = skeleton_data.iloc[:,:75]
                
                p_0 = np.copy(p_0)                
                p_0 = p_0.reshape([-1,25,3])
                t_0 = p_0.shape[0]
                if t_0>16:                   # sample the range from crop size of [16,t_0]
                    ratio = np.random.uniform(1,t_0/16)
                    l = int(16*ratio)
                    start = random.sample(range(t_0-l),1)[0]
                    end = start+l
                    p_0 = p_0[start:end,:,:]
                    p_0 = zoom1(p_0)
                elif t_0<16:
                    p_0 = zoom1(p_0)
                
                
                #Second person pose                
                p_1 = skeleton_data.iloc[:,75:]
                
                p_1 = np.copy(p_1)                
                p_1 = p_1.reshape([-1,25,3])
                t_1 = p_1.shape[0]
                if t_1 >16:  
                    ratio = np.random.uniform(1,t_1/16)
                    l = int(16*ratio)
                    start = random.sample(range(t_1-l),1)[0]
                    end = start+l
                    p_1 = p_1[start:end,:,:]
                    p_1 = zoom1(p_1)
                elif t_1 <16:
                    p_1 = zoom1(p_1)

                # randomly mirror augmentation 
                # since two persions' postion could be switched
                if np.random.choice([0,1],1): 
                    p_0, p_1 = mirror(p_0,p_1)

                #Calculate the temporal difference
                p_0_diff = p_0[1:,:,:]-p_0[:-1,:,:]
                #print(p_0[1:,:,:])
                p_0_diff = np.concatenate((p_0_diff,np.expand_dims(p_0_diff[-1,:,:],axis=0)))
                p_1_diff = p_1[1:,:,:]-p_1[:-1,:,:]
                p_1_diff = np.concatenate((p_1_diff,np.expand_dims(p_1_diff[-1,:,:],axis=0)))

                
                X_0.append(p_0)
                X_1.append(p_0_diff)
                X_2.append(p_1)
                X_3.append(p_1_diff)
                
                label = np.zeros(6)
                label[i-1] = 1
                Y.append(label)
                
   
    X_0 = np.stack(X_0) 
    X_1 = np.stack(X_1)
    X_2 = np.stack(X_2)
    X_3 = np.stack(X_3)
    Y = np.stack(Y) 
    
    
    history = model2.fit([X_0,X_1,X_2,X_3],Y,batch_size=32,epochs=1,verbose=True,shuffle=True)
    
    if not (e+1)%50:
        lr *= 0.8
        adam = optimizers.Adam(lr)
        model2.compile(adam, loss='mean_squared_error')
    
    print("===================================================================")
    

Epoch 1/1
74/74 [==============================] - 3s 42ms/step - loss: 0.2926
Epoch 1/1
74/74 [==============================] - 0s 687us/step - loss: 0.2207
Epoch 1/1
74/74 [==============================] - 0s 660us/step - loss: 0.1927
Epoch 1/1
74/74 [==============================] - 0s 647us/step - loss: 0.1657
Epoch 1/1
74/74 [==============================] - 0s 647us/step - loss: 0.1511
Epoch 1/1
74/74 [==============================] - 0s 647us/step - loss: 0.1339
Epoch 1/1
74/74 [==============================] - 0s 674us/step - loss: 0.1206
Epoch 1/1
74/74 [==============================] - 0s 647us/step - loss: 0.1111
Epoch 1/1
74/74 [==============================] - 0s 647us/step - loss: 0.1076
Epoch 1/1
74/74 [==============================] - 0s 647us/step - loss: 0.0986
Epoch 1/1
74/74 [==============================] - 0s 647us/step - loss: 0.0994
Epoch 1/1
74/74 [==============================] - 0s 660us/step - loss: 0.1005
Epoch 1/1
74/74 [========================

74/74 [==============================] - 0s 660us/step - loss: 0.0151
Epoch 1/1
74/74 [==============================] - 0s 660us/step - loss: 0.0135
Epoch 1/1
74/74 [==============================] - 0s 633us/step - loss: 0.0167
Epoch 1/1
74/74 [==============================] - 0s 647us/step - loss: 0.0153
Epoch 1/1
74/74 [==============================] - 0s 647us/step - loss: 0.0146
Epoch 1/1
74/74 [==============================] - 0s 660us/step - loss: 0.0080
Epoch 1/1
74/74 [==============================] - 0s 660us/step - loss: 0.0135
Epoch 1/1
74/74 [==============================] - 0s 647us/step - loss: 0.0161
Epoch 1/1
74/74 [==============================] - 0s 660us/step - loss: 0.0106
Epoch 1/1
74/74 [==============================] - 0s 647us/step - loss: 0.0085
Epoch 1/1
74/74 [==============================] - 0s 660us/step - loss: 0.0066
Epoch 1/1
74/74 [==============================] - 0s 647us/step - loss: 0.0105
Epoch 1/1
74/74 [==============================] -

74/74 [==============================] - 0s 633us/step - loss: 0.0032
Epoch 1/1
74/74 [==============================] - 0s 633us/step - loss: 0.0060
Epoch 1/1
74/74 [==============================] - 0s 633us/step - loss: 0.0037
Epoch 1/1
74/74 [==============================] - 0s 620us/step - loss: 0.0102
Epoch 1/1
74/74 [==============================] - 0s 633us/step - loss: 0.0071
Epoch 1/1
74/74 [==============================] - 0s 633us/step - loss: 0.0053
Epoch 1/1
74/74 [==============================] - 0s 620us/step - loss: 0.0039
Epoch 1/1
74/74 [==============================] - 0s 647us/step - loss: 0.0043
Epoch 1/1
74/74 [==============================] - 0s 633us/step - loss: 0.0028
Epoch 1/1
74/74 [==============================] - 0s 660us/step - loss: 0.0047
Epoch 1/1
74/74 [==============================] - 0s 647us/step - loss: 0.0037
Epoch 1/1
74/74 [==============================] - 0s 674us/step - loss: 0.0022
Epoch 1/1
74/74 [==============================] -

74/74 [==============================] - 0s 633us/step - loss: 0.0031
Epoch 1/1
74/74 [==============================] - 0s 633us/step - loss: 0.0046
Epoch 1/1
74/74 [==============================] - 0s 647us/step - loss: 0.0018
Epoch 1/1
74/74 [==============================] - 0s 647us/step - loss: 0.0021
Epoch 1/1
74/74 [==============================] - 0s 660us/step - loss: 0.0019
Epoch 1/1
74/74 [==============================] - 0s 687us/step - loss: 0.0056
Epoch 1/1
74/74 [==============================] - 0s 660us/step - loss: 0.0038
Epoch 1/1
74/74 [==============================] - 0s 647us/step - loss: 6.8901e-04
Epoch 1/1
74/74 [==============================] - 0s 647us/step - loss: 0.0022
Epoch 1/1
74/74 [==============================] - 0s 647us/step - loss: 0.0016
Epoch 1/1
74/74 [==============================] - 0s 633us/step - loss: 0.0036
Epoch 1/1
74/74 [==============================] - 0s 620us/step - loss: 0.0022
Epoch 1/1
74/74 [=============================

74/74 [==============================] - 0s 633us/step - loss: 0.0034
Epoch 1/1
74/74 [==============================] - 0s 647us/step - loss: 0.0023
Epoch 1/1
74/74 [==============================] - 0s 647us/step - loss: 6.7699e-04
Epoch 1/1
74/74 [==============================] - 0s 647us/step - loss: 0.0025
Epoch 1/1
74/74 [==============================] - 0s 633us/step - loss: 0.0023
Epoch 1/1
74/74 [==============================] - 0s 633us/step - loss: 3.2386e-04
Epoch 1/1
74/74 [==============================] - 0s 647us/step - loss: 7.7425e-04
Epoch 1/1
74/74 [==============================] - 0s 633us/step - loss: 0.0014
Epoch 1/1
74/74 [==============================] - 0s 647us/step - loss: 5.1073e-04
Epoch 1/1
74/74 [==============================] - 0s 647us/step - loss: 4.4334e-04
Epoch 1/1
74/74 [==============================] - 0s 647us/step - loss: 0.0015
Epoch 1/1
74/74 [==============================] - 0s 633us/step - loss: 0.0013
Epoch 1/1
74/74 [=============

Epoch 1/1
74/74 [==============================] - 0s 633us/step - loss: 1.0466e-04
Epoch 1/1
74/74 [==============================] - 0s 660us/step - loss: 0.0041
Epoch 1/1
74/74 [==============================] - 0s 633us/step - loss: 0.0047
Epoch 1/1
74/74 [==============================] - 0s 633us/step - loss: 0.0055
Epoch 1/1
74/74 [==============================] - 0s 633us/step - loss: 8.6948e-04
Epoch 1/1
74/74 [==============================] - 0s 633us/step - loss: 4.8443e-04
Epoch 1/1
74/74 [==============================] - 0s 633us/step - loss: 0.0016
Epoch 1/1
74/74 [==============================] - 0s 633us/step - loss: 4.7143e-04
Epoch 1/1
74/74 [==============================] - 0s 633us/step - loss: 6.9135e-05
Epoch 1/1
74/74 [==============================] - 0s 660us/step - loss: 0.0016
Epoch 1/1
74/74 [==============================] - 0s 660us/step - loss: 3.6959e-04
Epoch 1/1
74/74 [==============================] - 0s 633us/step - loss: 9.8984e-04
Epoch 1/1
74

Epoch 1/1
74/74 [==============================] - 0s 633us/step - loss: 0.0020
Epoch 1/1
74/74 [==============================] - 0s 633us/step - loss: 6.9233e-04
Epoch 1/1
74/74 [==============================] - 0s 647us/step - loss: 0.0012
Epoch 1/1
74/74 [==============================] - 0s 647us/step - loss: 0.0029
Epoch 1/1
74/74 [==============================] - 0s 647us/step - loss: 9.1001e-04
Epoch 1/1
74/74 [==============================] - 0s 620us/step - loss: 0.0012
Epoch 1/1
74/74 [==============================] - 0s 647us/step - loss: 4.1537e-04
Epoch 1/1
74/74 [==============================] - 0s 660us/step - loss: 0.0035
Epoch 1/1
74/74 [==============================] - 0s 633us/step - loss: 5.8527e-04
Epoch 1/1
74/74 [==============================] - 0s 633us/step - loss: 3.3976e-04
Epoch 1/1
74/74 [==============================] - 0s 647us/step - loss: 3.7724e-04
Epoch 1/1
74/74 [==============================] - 0s 674us/step - loss: 0.0027
Epoch 1/1
74/74 

74/74 [==============================] - 0s 620us/step - loss: 1.0830e-04
Epoch 1/1
74/74 [==============================] - 0s 660us/step - loss: 0.0058
Epoch 1/1
74/74 [==============================] - 0s 647us/step - loss: 6.3954e-04
Epoch 1/1
74/74 [==============================] - 0s 647us/step - loss: 3.5254e-04
Epoch 1/1
74/74 [==============================] - 0s 633us/step - loss: 1.9939e-04
Epoch 1/1
74/74 [==============================] - 0s 633us/step - loss: 4.0321e-04
Epoch 1/1
74/74 [==============================] - 0s 633us/step - loss: 1.6474e-04
Epoch 1/1
74/74 [==============================] - 0s 647us/step - loss: 0.0023
Epoch 1/1
74/74 [==============================] - 0s 647us/step - loss: 0.0013
Epoch 1/1
74/74 [==============================] - 0s 647us/step - loss: 6.0135e-04
Epoch 1/1
74/74 [==============================] - 0s 647us/step - loss: 1.3392e-04
Epoch 1/1
74/74 [==============================] - 0s 647us/step - loss: 1.4270e-04
Epoch 1/1
74/7

# Test model on X,Y,Z on UMassd-Kinect Dataset

In [ ]:
def zoom1(p):
    l = p.shape[0]
    #print(l)
    p_new = np.empty([16,25,3]) 
    #print(p_new)
    for m in range(25):
        for n in range(3):
            #print(m)
            #print(n)
            #print(p[:,m,n])
            p_new[:,m,n] = inter.zoom(p[:,m,n],16/l)[:16]
    return p_new

i = -1
X_TEST_0 = []
X_TEST_1 = []
X_TEST_2 = []
X_TEST_3 = []
Y_TEST = []

for root, dirs, files in os.walk("data/test"):
        if(i < 6):
            i = i+1
        for file in files:
            if file.endswith(".txt"):
                skeleton_data = pd.read_csv(os.path.join(root, file), sep=" ", header=None)
                p_0 = skeleton_data.iloc[:,:75]
                              
                p_0 = np.copy(p_0)                
                p_0 = p_0.reshape([-1,25,3])
                t_0 = p_0.shape[0]
                # if the number of frame is more than 20, crop by scale 0.9, then rescale by interploration again
                if t_0>=20: 
                    p_0 = p_0[int(t_0*0.05):int(t_0*0.95),:,:]
                    p_0 = zoom1(p_0)
                elif t_0<20:
                    p_0 = zoom1(p_0)
                p_0_diff = p_0[1:,:,:]-p_0[:-1,:,:]
                p_0_diff = np.concatenate((p_0_diff,np.expand_dims(p_0_diff[-1,:,:],axis=0)))
                
                
                p_1 = skeleton_data.iloc[:,75:]                           
                
                p_1 = np.copy(p_1)
                p_1 = p_1.reshape([-1,25,3])
                t_1 = p_1.shape[0]
                if t_1 >=20:  
                    p_1 = p_1[int(t_1*0.05):int(t_1*0.95),:,:]
                    p_1 = zoom1(p_1)
                elif t_1 <20:
                    p_1 = zoom1(p_1)
                p_1_diff = p_1[1:,:,:]-p_1[:-1,:,:]
                p_1_diff = np.concatenate((p_1_diff,np.expand_dims(p_1_diff[-1,:,:],axis=0)))

                X_TEST_0.append(p_0)
                X_TEST_1.append(p_0_diff)
                X_TEST_2.append(p_1)
                X_TEST_3.append(p_1_diff)
               
                label = np.zeros(6)
                label[i-1] = 1
                #print(label)
                Y_TEST.append(label)

X_TEST_0 = np.stack(X_TEST_0)
X_TEST_1 = np.stack(X_TEST_1)
X_TEST_2 = np.stack(X_TEST_2)
X_TEST_3 = np.stack(X_TEST_3)
Y_TEST = np.stack(Y_TEST)

Y_pred = model2.predict([X_TEST_0,X_TEST_1,X_TEST_2,X_TEST_3])

print('Predict labels:     ',np.argmax(Y_pred,axis=1))
print('Ground truth labels:',np.argmax(Y_TEST,axis=1))

# Calculating the Accuracy of the model

In [ ]:
predicted_true = 0
predicted_labels = np.argmax(Y_pred,axis=1)
ground_truth_labels = np.argmax(Y_TEST,axis=1)
for i in range(len(predicted_labels)):
    if(predicted_labels[i] == ground_truth_labels[i]):
        predicted_true +=1
print(predicted_true/len(predicted_labels)*100)